In [12]:
import openml
from sklearn.model_selection import train_test_split
from test_datasets.dataset_getters import get_pc4_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from tabpfgen import TabPFGen
from tabpfgen.visuals import visualize_classification_results
from sklearn.datasets import load_breast_cancer
import torch
import pandas as pd

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [4]:
train, test = get_pc4_dataset()

In [6]:
print(train.columns)

Index(['LOC_BLANK', 'BRANCH_COUNT', 'CALL_PAIRS', 'LOC_CODE_AND_COMMENT',
       'LOC_COMMENTS', 'CONDITION_COUNT', 'CYCLOMATIC_COMPLEXITY',
       'CYCLOMATIC_DENSITY', 'DECISION_COUNT', 'DECISION_DENSITY',
       'DESIGN_COMPLEXITY', 'DESIGN_DENSITY', 'EDGE_COUNT',
       'ESSENTIAL_COMPLEXITY', 'ESSENTIAL_DENSITY', 'LOC_EXECUTABLE',
       'PARAMETER_COUNT', 'HALSTEAD_CONTENT', 'HALSTEAD_DIFFICULTY',
       'HALSTEAD_EFFORT', 'HALSTEAD_ERROR_EST', 'HALSTEAD_LENGTH',
       'HALSTEAD_LEVEL', 'HALSTEAD_PROG_TIME', 'HALSTEAD_VOLUME',
       'MAINTENANCE_SEVERITY', 'MODIFIED_CONDITION_COUNT',
       'MULTIPLE_CONDITION_COUNT', 'NODE_COUNT',
       'NORMALIZED_CYLOMATIC_COMPLEXITY', 'NUM_OPERANDS', 'NUM_OPERATORS',
       'NUM_UNIQUE_OPERANDS', 'NUM_UNIQUE_OPERATORS', 'NUMBER_OF_LINES',
       'PERCENT_COMMENTS', 'LOC_TOTAL', 'c'],
      dtype='object')


# Test quality

In [21]:
generator = TabPFGen(n_sgld_steps=200, device=device)

X, y = train.drop("c", axis=1), train["c"].to_list()

X_synth, y_synth = generator.generate_classification(
    X, y,
    n_samples=X.shape[0],
    balance_classes=True,
)


Step 0/200
Step 100/200


In [22]:
clf = RandomForestClassifier().fit(X, y)
print("Train AUC:", roc_auc_score(y, clf.predict_proba(X)[:, 1]))
clf = RandomForestClassifier().fit(X_synth, y_synth)
print("Train AUC:", roc_auc_score(y, clf.predict_proba(X)[:, 1]))

Train AUC: 0.9999201419192179
Train AUC: 0.9566570266554866


/home/mikic202/miniconda3/envs/pt/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
